In [42]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import confusion_matrix, accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/amolsingh/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
ds = pd.read_csv("./labeled_data.csv")
ds.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [8]:
ds = ds[['class', 'tweet']]
y = ds.iloc[:, :-1].values

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [11]:
ohv = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
y = np.array(ohv.fit_transform(y))

In [13]:
y_df = pd.DataFrame(y)
y_hate = np.array(y_df[0])

In [20]:
#function from https://www.kaggle.com/code/pranjalpatil0205/hate-offensive-language?scriptVersionId=113248567&cellId=35
import re
import nltk
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [22]:
ds["cleaned_tweet"] = ds["tweet"].apply(clean_text)
cleaned = ds.cleaned_tweet.values.tolist()

[' rt mayasolov woman shouldnt complain clean hous amp man alway take trash', ' rt  boy dat coldtyga dwn bad cuffin dat hoe  place', ' rt urkindofbrand dawg rt  ever fuck bitch start cri confus shit', ' rt cganderson vivabas look like tranni', ' rt shenikarobert shit hear might true might faker bitch told ya ', 'tmadisonx shit blow meclaim faith somebodi still fuck hoe ', 'brighterday sit hate anoth bitch  got much shit go', ' caus im tire big bitch come us skinni ', ' amp might get ya bitch back amp that ', ' rhythmixx hobbi includ fight mariambitch', ' keek bitch curv everyon  lol walk convers like smh', ' murda gang bitch gang land ', ' hoe smoke loser   yea  go ig', ' bad bitch thing like ', ' bitch get ', ' bitch nigga miss ', ' bitch plz whatev ', ' bitch love ', ' bitch get cut everyday b ', ' black bottl amp bad bitch ', ' broke bitch cant tell noth ', ' cancel bitch like nino ', ' cant see hoe wont chang ', ' fuck bitch dont even suck dick   kermit video bout fuck ig', ' got y

In [23]:
print(len(cleaned))

24783


In [47]:
cv = CountVectorizer()
X = cv.fit_transform(cleaned).toarray()

In [49]:
tfid = TfidfVectorizer()
X = tfid.fit_transform(cleaned).toarray()


In [50]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y_hate, test_size = 0.20, random_state = 0)

In [52]:
classifier_dt = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier_dt.fit(X_train, y_train)


DecisionTreeClassifier(criterion='entropy', random_state=0)

In [55]:
import pickle
pickle.dump(classifier_dt, open("./TFid_DT.pickle", "wb"))

In [ ]:
loaded = pickle.load(open("./TFid_DT.pickle", "rb"))

In [ ]:
loaded.predict("hello")

In [53]:
#confusion matrix
y_pred_dt = classifier_dt.predict(X_test)
cm = confusion_matrix(y_test, y_pred_dt)
print(cm)

[[4542  136]
 [ 202   77]]


In [54]:
dt_score = accuracy_score(y_test, y_pred_dt)
print("Decision Tree Accuracy: ", dt_score)

Decision Tree Accuracy:  0.9318135969336292
